# 概要
- sklearn標準ライブラリを使用
- ボストンの犯罪データをpystanの線形回帰で分析

# ライブラリ準備

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib 
plt.style.use("ggplot")
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

import seaborn as sns

# 設定
import warnings
warnings.filterwarnings('ignore') # warningが出ないように設定
pd.set_option("display.max_rows", None) # pandasの表示上限をなくす
pd.set_option("display.max_columns", None) # pandasの表示上限をなくす

/Users/yoshidayuusei/.pyenv/versions/3.6.5/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/yoshidayuusei/.pyenv/versions/3.6.5/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# データ

## データの準備

In [3]:
from sklearn.datasets import load_boston
data = load_boston()
X = data.data
y = data.target

In [4]:
X = pd.DataFrame(X, columns=data.feature_names)
y = pd.DataFrame(y)

In [5]:
X.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


## データの分割

In [48]:
from sklearn.model_selection import train_test_split
X_train,  X_test, y_train, y_test = train_test_split(X.iloc[:,0:4], y, test_size=0.2, random_state=0)

# pystan利用

## データの定義

In [49]:
stan_data = {"N":X_train.shape[0],"M":X_train.shape[1],"X":X_train,"Y":y_train}

## モデルの定義

In [50]:
stan_model =""" 

data  {
    int <lower=0>N;
    int <lower=0>M;
    matrix[N, M] X;
    int<lower=0, upper=1> Y[N];
    
}

parameters{
    real beta_0;
    vector [M] beta;
    real <lower=0>sigma;
}

model{
    for (i in 1:N)
        Y[i] ~ normal(beta_0 + dot_product(X[i], beta), sigma);
}
""" 


## モデルコンパイル

In [51]:
from pystan import StanModel

stm = StanModel(model_code=stan_model)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_65504ff9703f8b0f34ba1c310e65c4aa NOW.


## MCMC

In [53]:
n_itr = 5000
n_warmup = 1000
chains = 2

# サンプリングの実行
fit = stm.sampling(data=, iter=n_itr, chains=chains, n_jobs=-1, warmup=n_warmup, algorithm="NUTS", verbose=False)

SyntaxError: invalid syntax (<ipython-input-53-2abd0f19e597>, line 6)

In [ ]:
print(fit)